# スタックとヒープを知る

- 2018/10/11 勉強会

# アジェンダ

1. 調べようと思ったきっかけ
1. メモリ領域概観
2. スタックについて
3. ヒープについて
4. まとめ
5. 参考

# 調べようと思ったきっかけ

- スタックとヒープわかってないとRustわけわかめになる

# メモリについて

- メモリは1Byte(8bit)ごとにアドレスが振られている
- 実行ファイルにコンパイルすると変数名はアドレス値に変換される
    - アドレス値は16進数
- 1つのプロセスは1つのメモリ空間と複数のスレッドを持つ
- 1つのスレッドは１つの連続したメモリ領域のスタックを持つ

# メモリ領域概観

- メモリ領域の「サイズ」はビルド時に決定する
- スタックとヒープに格納されるデータは動的に決まるので実行するまでどこを使うか決まらない
    - だから、下3つ以外の部分を上下両端から格納していく

![](https://he-s3.s3.amazonaws.com/media/uploads/383f472.png)

## 5つのメモリ領域

- 置かれるアドレスが
    - 実行時に決まるものが「動的」
    - コンパイル時に決まるものが「静的」

---
### 静的な領域
- text
    - プログラムの内、機械語の部分を置く
- data
    - 初期値のあるグローバル変数を置く
- bss
    - 初期値のないグローバル変数を置く
    - Block Started by Symbolの略
    - [参考リンク](http://www.ertl.jp/~takayuki/readings/info/no03.html)
    
### 動的な領域
- heap
    - プログラムのデータを置く
- stack
    - 関数の引数やローカル変数を置く
    - 動的にサイズが確保されるものを置く

## アドレスをGoで確認してみる

#### 複数の変数を宣言してアドレスを出力する

- これらの値はスタック領域に入る

```
package main

func main() {
	a, b, c := 11, 12, 13
	println(&a) // アドレスを出力
	println(&b)
	println(&c)
}

```

#### 出力結果
```
> go run address.go
0xc000040780
0xc000040778
0xc000040770 // 8bitずつ減っている
```

コンパイル(リンク)時に置かれるアドレスが決まる  → 静的
実行時に置かれるアドレスが決まる → 動的

--> スタック領域は、静的に確保される。スタック領域に置かれるデータは、動的に配置される。


- １つのプロセスは1つのメモリ空間と複数のスレッドを持つ
- 1つのスレッドは１つの連続したメモリ領域のスタックを持つ
- つまり、1つのプロセスのメモリ空間の中には複数のスタック用のメモリ領域がある


## メモリの領域

- なんで分かれているのか
    - メモリを使用する変数において、サイズが明確なものと、そうでないものがあるから
    - 変数の値やアドレスなどのサイズが分かっている値を格納するときはスタックを利用する
    - 動的にメモリを確保するものはヒープを利用する
    

    
- 参考
- https://keens.github.io/blog/2017/04/30/memoritosutakkutohi_puto/
- http://th0x4c.github.io/blog/2012/10/10/os-virtual-memory-map/


- スタックもヒープも両方RAM上のどこかのセクションの一部に属する
    - https://www.uquest.co.jp/embedded/learning/lecture11.html


- しつもん　https://teratail.com/questions/151094

Cの場合は以下の4つの領域に分けられる   
- プログラム領域
- 静的領域
- ヒープ領域
- スタック領域



# スタックについて

- 積み上げる、堆積物という意味
    - きれいに整然と積み上げていくイメージ
- LIFO

![](https://www.tutorialspoint.com/data_structures_algorithms/images/stack_representation.jpg)

- 【画像引用元】[Data Structures and Algorithms Stack](https://www.tutorialspoint.com/data_structures_algorithms/stack_algorithm.htm)

## スタックについて

- コンパイラやOSが自動で割り当てる領域
    - 領域の解放も自動的にやってくれる
    - 楽
- メモリ割り当てが高速
    - ローカル変数は全て事前にわかるので一度で割り当てられる
    - ヒープ割り当てより2~3桁速い(要出典)


## コードでの解釈

- スコープが入れ子になっているとき、内側のスコープで使うデータ領域は後から確保し、先に解放される
    
![](https://ufcpp.net/media/ufcpp2000/computer/fig/Essential/Stack.png)

- わかりやすい図解
    - [スタックメモリとヒープメモリ](http://article.higlabo.com/ja/stack_and_heap.html)
    - 【画像引用元】[メモリ管理 - コンピュータの基礎知識 | ++C++; // 未確認飛行 C](https://ufcpp.net/study/computer/MemoryManagement.html)

## 気をつけること

- スタックオーバーフロー
    - スタックのメモリのサイズを超えてpushすること
- スタックアンダーフロー
    - スタックが空の状態でさらにpopしようとすること
- スコープを抜ければスタック領域が強制的に解放される
    - ポインタがスタック領域を指すときは解放後にアクセスしないようにする
    
    


## スタック

- CPUはスタック領域の一番上のアドレスをスタックポインタレジスタ(SPレジスタ)に保持している(?)
    - push,popするとSPレジスタが移動する
    - https://www.uquest.co.jp/embedded/learning/lecture07-1.html

- 比較的サイズが小さくて、特定の関数内でしか使わない変数に向いている
    - 不必要に大きすぎると、使われない領域が無駄になる
    - なんで？
    
- 予めスコープが明確にわかっているデータに対してしか利用できない
- ローカル変数は全て事前にわかっているので、メモリを一度で確保できるの高速
    - アロケート時の速度の話
- 一般的にはスタックに使うためのメモリ領域はそれほど大きく取らない
    - 大きなデータ領域を一気に確保したい場合は向いていない
- 使用できるサイズは決まっているので、使いすぎるとスタックオーバーフローする


### 関連
- 逆ポーランド記法
    1. 数値だったらスタックにpush
    2. 演算子が来たら2回pop
    3. 演算したら答えをスタックにpush
    4. = がきたら1回popして答えとする
- スタックオーバーフロー
    - スタックメモリのサイズを超えてpushすること
- スタックアンダーフロー
    - スタックが空の状態でさらにpopしようとすること
    - Cとかで書いてると起こりえないが、アセンブラ言語とかなら起こりうる
    - https://www.uquest.co.jp/embedded/learning/lecture08.html
- バッファ・オーバーフロー
- メモリリーク
- GC

# ヒープについて

- 塊、山積みという意味
    - 乱雑に山積みするイメージ

![](https://craftofcoding.files.wordpress.com/2015/12/stackmemory4.jpg)

## ヒープについて

- 動的に割り当てたり、開放できる領域
- メモリの確保と解放を自分自身で行う必要がある
    - サイズは動的に指定できる
    - 確保、解放する順番は任意
- 低速
    - アロケート時は空いている領域を検索する
    - 読み込み時もポインタアドレスから参照する

## 用途

- データを必要とするスコープがはっきりしないとき
    - さまざまな箇所から参照される場合など
- ファイルを読み込むとき
    - ファイルサイズがわからないから

## ヒープ


- ポインタはヒープに確保される(?)
## 用途

- データのサイズが大規模な場合
    - なんで？

### 関連

- 双方向リンク構造
    - 空きブロックどうしがそれぞれ双方向のリンクで結ばれ、これらのリンク途中へのノードの挿入や除去の操作が行われる
    - https://www.ipa.go.jp/security/awareness/vendor/programmingv2/contents/c007.html



## C言語では

- malloc()でメモリアロケーション
- free()で解放

------

- 擬似コード
    - 【参考】[ヒープとスタック | 学校では教えてくれないこと | [技術コラム集]組込みの門 | ユークエスト株式会社](https://www.uquest.co.jp/embedded/learning/lecture16.html)

```
ファイルオープン;
file_sizeを取得;
buf = malloc(file_size);
bufにファイル内容を読み出し;
free(buf);
```

    


## C言語

- malloc()は`void*`型のポインタを返すので、キャストして使うことが多い
- `p = (char *) malloc(n);` // n byteのchar型のメモリ領域を確保
- `malloc(n)` (`p`)はnバイトの領域を確保した先頭のアドレスを返す
- free(p)で解放

- Cの場合の配列のポインタは先頭要素のアドレスを示す
- array[1]などは、配列名のアドレス+要素番号のメモリを参照ということになる


## C++では

- new演算子でメモリアロケーション
- delete演算子で解放

- コンストラクタ、デストラクタがある辺りが若干違う
    - http://book.geocities.jp/muu5muu/doc1/sample6.html
    - https://brain.cc.kogakuin.ac.jp/~kanamaru/lecture/C++2/09/09-02.html





## Rustでは

- `Box<T>型`をつかうとヒープ上にアロケートできる
    - `i32`型の値5をxに格納
    - `let x: Box<i32> = Box::new(5)`
- [他の方法](http://mmi.hatenablog.com/entry/2017/08/06/230823)

## ちなみにGoでは

- スタックとヒープを気にする必要がない
    - コンパイラがよしなに行ってくれる
    - newしても必要がなければスタックにアロケートする
    - 【参考】[golangではスタックとヒープを気にする必要が無い - Qiita](https://qiita.com/rookx/items/a1e3d057a0ed71424094)

## 気をつけること

- 確保、解放を頻繁にやりすぎるとフラグメンテーションの原因になる
    - 参照の局所性が失われ、キャッシュのヒット率低下が起こり、処理性能が悪化する
    - GC後、使用領域を一箇所にまとめるコンパクションという処理を行うこともある
- 確保に失敗することもある
    - 物理的な空きメモリがないとき
    - アドレス空間に空きがないとき

## 気をつけること

- 物理的な空きメモリが無いとき
    - 最近のPCでは仮想メモリも作るのでアンマリ起こらない(?)
- アドレス空間に空きがないとき
    - 32bit環境ならポインタが32bitしかない
    - 4GB?
    - https://theolizer.com/cpp-school1/cpp-school1-11/

# まとめ

- メモリにはいくつかの領域がある
- スタックはLIFOでスコープが寿命
- ヒープは自分で確保と解放を行う

# 参考

- [ヒープとスタック | 学校では教えてくれないこと | [技術コラム集]組込みの門 | ユークエスト株式会社](https://www.uquest.co.jp/embedded/learning/lecture16.html)
- [スタックメモリとヒープメモリ](http://article.higlabo.com/ja/stack_and_heap.html)
- [メモリ管理 - コンピュータの基礎知識 | ++C++; // 未確認飛行 C](https://ufcpp.net/study/computer/MemoryManagement.html)
- [メモリとスタックとヒープとプログラミング言語 | κeenのHappy Hacκing Blog](https://keens.github.io/blog/2017/04/30/memoritosutakkutohi_puto/)
- [malloc ライブラリのメモリ管理構造 | 技術文書 | 技術情報 | VA Linux Systems Japan株式会社](https://www.valinux.co.jp/technologylibrary/document/linux/malloc0001/)(未読)
- [ポインタと配列 - A Day In The Life](http://glassonion.hatenablog.com/entry/20090414/1239716083)

## メモリリーク

- ヒープを利用するときに、用済みのデータ領域を開放せずにゴミが溜まる状態

## 二重解法

- すでに解放済みの領域を再度開放しようとすること

## ガベージコレクション

- ヒープの(?)のメモリ領域を開放すること
- 有効な領域から参照されていないメモリ領域を開放する

- https://ufcpp.net/study/computer/MemoryManagement.html
- Mark and Serrp
    - スコープ(有効な領域)から辿って、参照されていない領域を探してGCをする「マーク&スイープ」方式というのがある
    - rubyのGC
    - jsも
    -  https://developer.mozilla.org/ja/docs/Web/JavaScript/Memory_Management#Mark-and-sweep_algorithm
- Copy GC方式
    - ヒープ領域を2つとって、1箇所にまとめてコピーしていきながらコンパクションとGCを同時に行う
    - 片一方(ヒープ領域の半分)が未使用になってしまう
    - Java,OCaml,HaskellのGC
- 世代別GC
    - ヒープ領域を複数の世代に分ける
    - ruby,javaなど
- 参照カウント方式
    - https://developer.mozilla.org/ja/docs/Web/JavaScript/Memory_Management#Reference-counting_garbage_collection
    - 値を参照するたびにカウントを増やし、使わなくなったら減らし、0になったらfree
    - python,phpのGC
    - jsも
    
    
- https://developer.mozilla.org/ja/docs/Web/JavaScript/Memory_Management
- 難しいのは割り当てられたメモリが、今後必要とされない」ことを保証すること
- > メモリーの断片が不要になって解放する必要があるかを決定するには、開発者による判断が必要なことが多いです。




# ポインタ

- ポインタはアドレス変数
- 変数のアドレスを格納する変数

- (Cの場合)、必ず以下の3ステップで用いる
    1. 宣言
    2. 値(アドレス)の設定
    3. 使用
    
    
- `*`を使って型を指定してポインタを宣言
    - ポインタの差すものはただのアドレスなので、それの値の型がわからないので型を指定
```
var ptr *int
```

- `&`を使ってアドレスを取得
    - アンパサンドレス
```
pointer = &n // 変数nのアドレス
```

- `*`アドレスから変数の中身を見る
```
*pointer
    --------←あどれす
------------←値
```

- アドレス自体は演算できない(Goは)
    - `// pointer = pointer + 1               `    
    - `// tmp/sandbox390357081/main.go:37:20: invalid operation: pointer + 1 (mismatched types *int and int)  `
    
    
- 関数宣言時の引数に`*`がついているときはなに？？？？？
- func hoge(*a) {} ←宣言時。このとき*a, aはそれぞれ何？
- hoge(&a) ←呼び出し時。アドレスを渡している?
- https://fisproject.jp/2014/01/%E3%80%90c%E8%A8%80%E8%AA%9E%E3%80%91%E3%83%9D%E3%82%A4%E3%83%B3%E3%82%BF%E3%81%A8%E3%83%A1%E3%83%A2%E3%83%AA%E3%81%AE%E8%A9%B1%E3%80%90%E3%83%92%E3%83%BC%E3%83%97%E3%80%91/
    
## ポインタのポインタ

- `**hoge`
- ポインタ変数のアドレスを格納する変数
- ポインタのアドレスを書き換えたいときに使う
- https://bituse.info/c/28
    
    
## 値渡しと参照渡し

- 関数の引数に値をセットすると、関数内で値がコピーされる(値渡し)

- 関数の引数にポインタをセットすると、関数内での変更は元の値にも影響がある(参照渡し)
- https://www.yoheim.net/blog.php?q=20170901

# Rustの話

- http://vividcode.hatenablog.com/entry/rust/introduction/2
- https://doc.rust-jp.rs/the-rust-programming-language-ja/1.6/book/the-stack-and-the-heap.html
- http://south37.hatenablog.com/entry/2014/12/14/Rust%E3%81%AB%E3%81%8A%E3%81%91%E3%82%8BBorrowing%E3%81%AE%E4%BB%95%E7%B5%84%E3%81%BF ←若干文法が古い
- RustにGCはない


- `&`をつけると参照になる
- 参照はいくつでも作れる
- https://qiita.com/cactaceae/items/2c70a9947364c60ec100
```
let v1 = vec![1, 2, 3];
let v2 = &v1;
let v3 = &v2;
println!("Hello, {}, {}, {} !", v1[0], v2[1], v3[2]); 
````

## ownership

- https://blog.waft.me/2017/09/24/rust-heap/
- C++ではコンパイルは通るが、実行すると落ちる
    - https://qiita.com/myhr47/items/262b8b8f1edd8097d03c
- http://mao-instantlife.hatenablog.com/entry/2016/05/05/Rust%E3%81%AE%E5%8B%89%E5%BC%B7%EF%BC%88%E3%81%9D%E3%81%AE%EF%BC%91%EF%BC%91%EF%BC%89
- Rustではムーブセマンティクスを通して1つの変数のみに所有権をもたせることで、リソースの二重解放を防いでいる

- 変数が開放されればデストラクタが呼ばれ、スタック、ヒープともに開放される
- 所有権があることによってデストラクトのタイミングを正確に把握できる
```
fn foo() {
    let v = vec![1, 2, 3];
}
```
- 新しいvectorはスタック上、各要素のためのスペースはヒープ上に確保
- スコープを抜ければ、vectorに関するリソースが解放される
    - スタック上のものは取り除かれ、ヒープ上のものものfreeされる

- https://qiita.com/kakaka19/items/2ba88ae0a148951d6f4c
    - http://ykomatsu.github.io/rust/book-ja/ownership.html

### そもそも束縛ってなに


- 以下の例の場合、「束縛されている値」は`vec![1,2,3]`のこと?
- `vec![1,2,3]`を名前`v`に束縛 ?
```
fn foo() {
    let v = vec![1, 2, 3];
}
```

## borrowing

- ownershipは移さずに値への参照を可能にする


## lifetime



# Goの話

## ポインタ

- https://qiita.com/Sekky0905/items/447efa04a95e3fec217f




- https://qiita.com/Sekky0905/items/c9cbda2498a685517ad0#%E3%83%9D%E3%82%A4%E3%83%B3%E3%82%BF
- `%p`でポインタ
```
fmt.Printf("ポインタ = %p", p)  // -> ポインタ = 0x1040a130

```

## mapとsliceの違い